In [2]:
!pip install pykrx

In [3]:
from pykrx import stock
import time
import pandas as pd
import OpenDartReader
import pandas as pd
from tqdm import tqdm
api_key = '6655d5a62b916e96636cab74f8e7e0b05692b825'
dart = OpenDartReader(api_key)

In [4]:
# KOSDAQ 종목코드 조회
stock_code = stock.get_market_ticker_list(date="20210520", market="KOSDAQ")
(stock_code[:6], len(stock_code))

(['060310', '054620', '265520', '211270', '035760', '051500'], 1503)

In [5]:
def asset_f(year, corp_code, report_code):
    fs = dart.finstate_all(corp=corp_code, bsns_year=year, fs_div='OFS', reprt_code=report_code) 
    try:
        equity = int(fs.loc[fs['sj_div'].isin(['BS']) & fs['account_id'].isin(['ifrs-full_Equity']), 'thstrm_amount'].replace(",", "")) # 당기자본(자본총계)
        liability = int(fs.loc[fs['sj_div'].isin(['BS']) & fs['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount'].replace(",", "")) # 당기부채(부채총계)
        assets = equity + liability # 자산총계
        return assets
    except:
        equity = int(fs.loc[fs['sj_div'].isin(['BS']) & fs['account_id'].isin(['ifrs_Equity']), 'thstrm_amount'].replace(",", "")) # 당기자본(자본총계)
        liability = int(fs.loc[fs['sj_div'].isin(['BS']) & fs['account_id'].isin(['ifrs_Liabilities']), 'thstrm_amount'].replace(",", "")) # 당기부채(부채총계)
        assets = equity + liability # 자산총계
        return assets

stock_code = stock.get_market_ticker_list() # 현재일자 기준 가장 가까운 영업일의 코스피 상장종목 리스트
res = pd.DataFrame()
for ticker in tqdm(stock_code[:4]):
    df1 = stock.get_market_ohlcv_by_date(fromdate="20210117", todate="20210520", ticker=ticker) # 기본
    df2 = stock.get_market_fundamental_by_date(fromdate="20210117", todate="20210520", ticker=ticker) # 계산값
    df = pd.concat([df1, df2], axis=1)
    df = df.assign(종목코드=ticker, 종목명=stock.get_market_ticker_name(ticker))
    res = pd.concat([res, df], axis=0)
    time.sleep(1)
res = res.reset_index()
res

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:08<00:00,  2.18s/it]


,날짜,시가,고가,저가,종가,거래량,BPS,PER,PBR,EPS,DIV,DPS,종목코드,종목명
0,2021-01-18,4525,4560,4430,4485,77316,6802,4.57,0.66,982,6.69,300,095570,AJ네트웍스
1,2021-01-19,4485,4495,4370,4465,106387,6802,4.55,0.66,982,6.72,300,095570,AJ네트웍스
2,2021-01-20,4460,4600,4435,4510,114901,6802,4.59,0.66,982,6.65,300,095570,AJ네트웍스
3,2021-01-21,4210,4280,4005,4175,1468118,6802,4.25,0.61,982,7.19,300,095570,AJ네트웍스
4,2021-01-22,4190,4245,4160,4210,216835,6802,4.29,0.62,982,7.13,300,095570,AJ네트웍스
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,2021-05-13,181000,184000,177000,180000,51020,40317,25.34,4.46,7103,1.33,2400,282330,BGF리테일
332,2021-05-14,181000,184000,177000,177500,25706,40317,24.99,4.40,7103,1.35,2400,282330,BGF리테일
333,2021-05-17,177500,181500,176000,180500,30384,40317,25.41,4.48,7103,1.33,2400,282330,BGF리테일
334,2021-05-18,183000,183500,180000,182000,21897,40317,25.62,4.51,7103,1.32,2400,282330,BGF리테일


In [6]:
res['분기'] = pd.PeriodIndex(res['날짜'], freq='Q')
res

,날짜,시가,고가,저가,종가,거래량,BPS,PER,PBR,EPS,DIV,DPS,종목코드,종목명,분기
0,2021-01-18,4525,4560,4430,4485,77316,6802,4.57,0.66,982,6.69,300,095570,AJ네트웍스,2021Q1
1,2021-01-19,4485,4495,4370,4465,106387,6802,4.55,0.66,982,6.72,300,095570,AJ네트웍스,2021Q1
2,2021-01-20,4460,4600,4435,4510,114901,6802,4.59,0.66,982,6.65,300,095570,AJ네트웍스,2021Q1
3,2021-01-21,4210,4280,4005,4175,1468118,6802,4.25,0.61,982,7.19,300,095570,AJ네트웍스,2021Q1
4,2021-01-22,4190,4245,4160,4210,216835,6802,4.29,0.62,982,7.13,300,095570,AJ네트웍스,2021Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,2021-05-13,181000,184000,177000,180000,51020,40317,25.34,4.46,7103,1.33,2400,282330,BGF리테일,2021Q2
332,2021-05-14,181000,184000,177000,177500,25706,40317,24.99,4.40,7103,1.35,2400,282330,BGF리테일,2021Q2
333,2021-05-17,177500,181500,176000,180500,30384,40317,25.41,4.48,7103,1.33,2400,282330,BGF리테일,2021Q2
334,2021-05-18,183000,183500,180000,182000,21897,40317,25.62,4.51,7103,1.32,2400,282330,BGF리테일,2021Q2


In [7]:
quater = {'Q1': '11013',
          'Q2': '11012',
          'Q3': '11014',
          'Q4': '11011'}
res['자산'] = ['' for i in range(len(res['분기']))]
for i in range(len(res['자산'])):
    year = str(res['분기'][i])[:4] # 년도 
    c_code = res['종목코드'][i] # 회사 코드
    q_code = quater[str(res['분기'][i])[-2:]] #분기 코드
    res['자산'][i] = asset_f(year, c_code, q_code)
res

<ipython-input-7-762e811de297>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['자산'][i] = asset_f(year, c_code, q_code)


,날짜,시가,고가,저가,종가,거래량,BPS,PER,PBR,EPS,DIV,DPS,종목코드,종목명,분기,자산
0,2021-01-18,4525,4560,4430,4485,77316,6802,4.57,0.66,982,6.69,300,095570,AJ네트웍스,2021Q1,1218314937498
1,2021-01-19,4485,4495,4370,4465,106387,6802,4.55,0.66,982,6.72,300,095570,AJ네트웍스,2021Q1,1218314937498
2,2021-01-20,4460,4600,4435,4510,114901,6802,4.59,0.66,982,6.65,300,095570,AJ네트웍스,2021Q1,1218314937498
3,2021-01-21,4210,4280,4005,4175,1468118,6802,4.25,0.61,982,7.19,300,095570,AJ네트웍스,2021Q1,1218314937498
4,2021-01-22,4190,4245,4160,4210,216835,6802,4.29,0.62,982,7.13,300,095570,AJ네트웍스,2021Q1,1218314937498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,2021-05-13,181000,184000,177000,180000,51020,40317,25.34,4.46,7103,1.33,2400,282330,BGF리테일,2021Q2,2496172938906
332,2021-05-14,181000,184000,177000,177500,25706,40317,24.99,4.40,7103,1.35,2400,282330,BGF리테일,2021Q2,2496172938906
333,2021-05-17,177500,181500,176000,180500,30384,40317,25.41,4.48,7103,1.33,2400,282330,BGF리테일,2021Q2,2496172938906
334,2021-05-18,183000,183500,180000,182000,21897,40317,25.62,4.51,7103,1.32,2400,282330,BGF리테일,2021Q2,2496172938906


## Asset 구하기

In [8]:
fs = dart.finstate_all(corp='095570', bsns_year='2021', fs_div='CFS', reprt_code=11013) 
fs['rcept_no'] = [i[:8] for i in fs['rcept_no']]
fs['rcept_no'] = pd.to_datetime(fs['rcept_no'])
fs['rcept_no'][0]

Timestamp('2021-05-17 00:00:00')

In [9]:
# 자본과 부채는 재무상태표에서 당기금액('thstrm_amount') 값을 가져오면 됨
equity = int(fs.loc[fs['sj_div'].isin(['BS']) & fs['account_id'].isin(['ifrs-full_Equity']), 'thstrm_amount'].replace(",", "")) # 당기자본(자본총계)
liability = int(fs.loc[fs['sj_div'].isin(['BS']) & fs['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount'].replace(",", "")) # 당기부채(부채총계)
assets = equity + liability # 자산총계
print(assets)


1574355448723


In [10]:
# 문제는 보고서가 언제 나왔는지 모른다는 것이다. 그러나 작년까지는 일단 나왔을 것이고, 그렇다면 우리는 그걸 바탕으로 구하면 된다. 

In [ ]:
# 예를 들어서 2019년 - 2020년의 데이터 즉 총 8분기의 데이터를 구한다고 생각해 보자 
# 한 종목 먼저 해보자
# 095570
# corp: 종목코드, bsns_year: 사업연도, reprt_code: 보고서코드(1분기보고서 : 11013, 반기보고서 : 11012, 3분기보고서 : 11014, 사업보고서 : 11011)

year = ['2019', '2020']
reprt_code = ['11013', '11012', '11014', '11011']

def asset_f(year, corp_code, report_code):
    fs = dart.finstate_all(corp=corp_code, bsns_year=year, fs_div='OFS', reprt_code=report_code) 
    
    try:
        equity = int(fs.loc[fs['sj_div'].isin(['BS']) & fs['account_id'].isin(['ifrs-full_Equity']), 'thstrm_amount'].replace(",", "")) # 당기자본(자본총계)
        liability = int(fs.loc[fs['sj_div'].isin(['BS']) & fs['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount'].replace(",", "")) # 당기부채(부채총계)
        assets = equity + liability # 자산총계
        return assets
    except:
        equity = int(fs.loc[fs['sj_div'].isin(['BS']) & fs['account_id'].isin(['ifrs_Equity']), 'thstrm_amount'].replace(",", "")) # 당기자본(자본총계)
        liability = int(fs.loc[fs['sj_div'].isin(['BS']) & fs['account_id'].isin(['ifrs_Liabilities']), 'thstrm_amount'].replace(",", "")) # 당기부채(부채총계)
        assets = equity + liability # 자산총계
        return assets

for i in reprt_code:
    print(asset_f('2020', '282330', i))
        


2128035497561


In [ ]:
fs = dart.finstate_all(corp='211270', bsns_year='2020', fs_div='CFS', reprt_code='11013')
fs['account_id'].isin(['ifrs-full_Equity']).sum()

In [ ]:
fs.loc[fs['sj_div'].isin(['BS']) & fs['account_id'].isin(['ifrs_Equity']), 'thstrm_amount']

In [ ]:
fs